# SiPM Showers

#### 1) Select events using a run's DST file -- > 2) Look at those events' waveforms

Here, I use this notebook to look at the waveforms of sodium events near the photoelectric peak

In [1]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2017-04-27 15:51:59


In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
import time
import tables as tb
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

from invisible_cities.database import load_db
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.reco.tbl_functions as tbl
from invisible_cities.reco.params import S12Params, ThresholdParams
from   invisible_cities.core.system_of_units_c import units
from invisible_cities.core.core_functions import in_range
from invisible_cities.core.mpl_functions import plot_pmt_waveforms, plot_pmt_signals_vs_time_mus, \
              plot_signal_vs_time_mus
from invisible_cities.reco.pmaps_functions import plot_s12

sys.path.append('/Users/alej/Desktop/Valencia/nextic/ICARO')
from icaro.core.event_pmaps     import EventPmaps, print_s12, print_s2si
from icaro.core.kdst_functions  import load_dst, event_rate, profile_and_fit, print_fit, chi2
from icaro.core.hst_functions   import labels, hist, doublehist, hist2d, pdf, scatter, profile_and_scatter,\
                                     doublescatter, covariance, reso, gausstext, plot_profile_histogram 

### Run Parameters

In [4]:
run_number =  3723

# consecutive! could do a little more work and make them not have to be consecutive
subruns     = ['025', '026', '027', '028', '029']

In [5]:
DataPMT = load_db.DataPMT(run_number)
DataSiPM = load_db.DataSiPM(run_number)
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

Define External SiPMs

In [6]:
nsipm = 1792
ext_t = 170 * units.mm
ext_sipms = np.where(np.sqrt(xs **2 + ys ** 2) > ext_t)[0]
mask            = np.ones_like(xs, dtype=bool)
mask[ext_sipms] = False
int_sipms = np.where(mask)[0]
assert       len(int_sipms) + len(ext_sipms)  == 1792
assert  list(set(int_sipms) | set(ext_sipms)) == list(range(nsipm))
print(len(int_sipms), len(ext_sipms))

912 880


### 1) HAVE ACCESS TO WVFMS FROM CONSECTIVE SUBRUNS

In [7]:
## consecutive meaning subrun 025, 026, 027... for ex

RWF_path = os.path.join(os.environ['IC_DATA'], 'LSC/wvfms/{}/'.format(run_number))
RWF_files = [RWF_path + 'dst_waves.gdcsnext.{}_{}.root.h5'.format(subrun, run_number) for subrun in subruns]
print(RWF_files)

['/Users/alej/Desktop/IC_DATA/LSC/wvfms/3723/dst_waves.gdcsnext.025_3723.root.h5', '/Users/alej/Desktop/IC_DATA/LSC/wvfms/3723/dst_waves.gdcsnext.026_3723.root.h5', '/Users/alej/Desktop/IC_DATA/LSC/wvfms/3723/dst_waves.gdcsnext.027_3723.root.h5', '/Users/alej/Desktop/IC_DATA/LSC/wvfms/3723/dst_waves.gdcsnext.028_3723.root.h5', '/Users/alej/Desktop/IC_DATA/LSC/wvfms/3723/dst_waves.gdcsnext.029_3723.root.h5']


### 2) FIND STARTING AND FINAL EVENT IN THESE SUBRUNS

In [8]:
startfile = tb.open_file(RWF_files[0],'r')
sev = startfile.root.Run.events[0][0]
startfile.close()

endfile = tb.open_file(RWF_files[-1],'r')
fev = endfile.root.Run.events[-1][0]
endfile.close()
print('These subruns include events in this (inclusive) range: ')
print(sev, fev)

These subruns include events in this (inclusive) range: 
4101 4920


In [9]:
ifile = os.path.join(os.environ['IC_DATA'],
                        'LSC/kdst/{}/dst_{}.root.h5'.format(run_number,run_number))
print("ifile:", ifile)
full = load_dst(ifile)

ifile: /Users/alej/Desktop/IC_DATA/LSC/kdst/3723/dst_3723.root.h5


## 3) Select events in dst in these subruns
also can make other cuts 

In [10]:
srs  = full[in_range(full.event, sev, fev)] # events from this subrun
srsc = srs [srs .nS2 ==1]                   # events with one S2
srsc = srsc[srsc.peak==0]                   # events with one S1
print(len(srs), len(srsc))
srsc

100 50


,event,time,peak,nS2,S1w,S1h,S1e,S1t,S2w,S2h,...,S2t,Nsipm,DT,Z,X,Y,R,Phi,Xrms,Yrms
545,4102,1.492619e+09,0,1,150.0,17.618677,87.857823,200100.0,19.996312,17215.431641,...,288462.500,30,88.362500,88.362500,-139.515474,-81.782375,161.718658,-2.611391,10.188607,10.138916
550,4108,1.492619e+09,0,1,100.0,10.593005,41.233344,200125.0,10.835000,5700.332520,...,368987.500,16,168.862500,168.862500,-194.816681,0.474872,194.817260,3.139155,8.516031,8.939149
551,4109,1.492619e+09,0,1,150.0,14.398573,72.112009,200125.0,19.191250,6103.070312,...,376837.500,16,176.712500,176.712500,-44.732554,-1.300757,44.751462,-3.112522,8.219298,8.762655
557,4266,1.492619e+09,0,1,175.0,19.453815,104.532571,200125.0,36.650000,14557.035156,...,341112.500,25,140.987500,140.987500,51.274244,3.167983,51.372018,0.061707,10.748890,9.728248
558,4267,1.492619e+09,0,1,175.0,22.656319,120.534444,200125.0,23.874906,11637.315430,...,376937.500,23,176.812500,176.812500,-37.333625,4.840414,37.646104,3.012659,9.358130,10.681424
559,4270,1.492619e+09,0,1,100.0,9.861050,42.495188,200125.0,13.676875,4892.188965,...,357337.500,15,157.212500,157.212500,-193.646266,-30.231318,195.991859,-2.986727,8.486188,8.247123
560,4271,1.492619e+09,0,1,100.0,8.465937,36.181121,200150.0,9.811656,10047.573242,...,301112.500,17,100.962500,100.962500,-159.846853,-29.529507,162.551556,-2.958916,9.129866,8.436586
563,4273,1.492619e+09,0,1,200.0,25.775980,143.176807,200150.0,38.860000,12594.197266,...,339987.500,24,139.837500,139.837500,12.556256,-3.233011,12.965798,-0.252008,10.803112,9.802220
564,4274,1.492619e+09,0,1,100.0,10.497008,41.978358,200100.0,10.969156,9348.986328,...,361362.500,20,161.262500,161.262500,107.414561,5.509940,107.555787,0.051251,8.986230,9.660477
565,4276,1.492619e+09,0,1,150.0,19.054533,93.369272,200100.0,19.048000,2599.987061,...,720037.500,8,519.937500,519.937500,-83.800201,83.641245,118.399035,2.357144,7.210822,7.565852


In [ ]:
s1par  = S12Params(tmin=  0*units.mus, tmax=640*units.mus, stride= 4, lmin= 6, lmax=   20, rebin=False)
s2par  = S12Params(tmin=645*units.mus, tmax=700*units.mus, stride=80, lmin=160, lmax=200000, rebin=True)
thr    = ThresholdParams(thr_s1=3*units.pes,  thr_s2=1*units.pes,
                         thr_MAU=3*units.adc, thr_sipm = .5 * units.pes,
                         thr_SIPM=30*units.adc)
epm = EventPmaps(run_number, s1par, s2par, thr, verbose=False)

In [ ]:
MEVTS          = 2000
nsipm          = 1792
npmt           = 11
region_len_mus = 700

Aswf     = np.zeros((nsipm, region_len_mus), dtype=np.float32)
Bswf     = np.zeros((nsipm, region_len_mus), dtype=np.float32)

Aswf_pmt     = np.zeros((npmt, int(region_len_mus * units.mus / 25)), dtype=np.float32)
Bswf_pmt     = np.zeros((npmt, int(region_len_mus * units.mus / 25)), dtype=np.float32)

Apes_pmt = []
Bpes_pmt = []
Apes     = []
Aext_pes = []
Aint_pes = []
At       = []

Bpes  = []
Bspes = []
Bfpes = []
Bt    = []

Bext_pes = []
Bint_pes = []

pevts = 0
for subrun in RWF_files:
    if pevts == MEVTS: break
    h5rwf = tb.open_file(subrun,'r')
    pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)
    NEVT, NPMT, PMTWL = pmtrwf.shape
    NEVT, NSIPM, SIPMWL = sipmrwf.shape
    print("""
            Number of events in file = {}
            Number of PMTs = {}
            PMTWL = {}
            Number of SiPMs = {}
            SiPMWL = {}
          """.format(NEVT, NPMT, PMTWL, NSIPM, SIPMWL))
    
    # Select desired waveforms
    assert(NEVT == h5rwf.root.Run.events[-1][0] - h5rwf.root.Run.events[0][0] + 1)
    ssev = h5rwf.root.Run.events[0][0]
    sfev = h5rwf.root.Run.events[-1][0]
    sr = srs[in_range(srs.event, ssev, sfev)]
    srevents = np.array(list(set(sr.event.values)))
    srinds = srevents - ssev
    spmtrwf = np.array(pmtrwf) [srinds]
    ssipmrwf= np.array(sipmrwf)[srinds]
    
    print('IC Na22 Candidates in this subrun: ', len(ssipmrwf))
    
    for evi in range(len(spmtrwf)):
        
        # Run IC
        epm.calibrated_pmt_and_csum(evi, spmtrwf)
        epm.calibrated_sipm(evi, ssipmrwf, calwf=True)
        epm.find_s1()
        epm.find_s2()
        epm.find_s2si()
        
        # Find A and B regions (ns)
        tbuf   = 1 * units.mus
        Atimes = np.array([epm.S1[0][0][-1] + tbuf, epm.S2[0][0][0] - tbuf], dtype=np.float32)
        Btimes = np.array([epm.S2[0][0][-1] + tbuf, 1300000         - tbuf], dtype=np.float32)
        Ai     = np.array(np.ceil(Atimes / units.mus), dtype=np.int32) # indices of sipm wvfm (same as time in mus)
        Bi     = np.array(np.ceil(Btimes / units.mus), dtype=np.int32)
        regA   = epm.sipm[:, Ai[0]: Ai[1]+1]
        regB   = epm.sipm[:, Bi[0]: Bi[1]+1]
        
        # Get cumulative waveform of PMTs (a kind of positive control)
        Ai_pmt   = np.array(Ai * units.mus / 25.0 * units.ns, dtype=np.int32)
        Bi_pmt   = np.array(Bi * units.mus / 25.0 * units.ns, dtype=np.int32)
        regA_pmt = epm.CAL_PMT[:, Ai_pmt[0]: Ai_pmt[1]+1]
        regB_pmt = epm.CAL_PMT[:, Bi_pmt[0]: Bi_pmt[1]+1]
        Aswf_pmt[:, :regA_pmt.shape[1]] += regA_pmt
        Bswf_pmt[:, :regB_pmt.shape[1]] += regB_pmt
        
        # Add A and B regions to Aswf and Bswf 
        Aswf[:, :regA.shape[1]] += regA
        Bswf[:, :regB.shape[1]] += regB
        
        # Record pes in these regions
        Apes.append(regA.sum(axis=1))
        Bpes.append(regB.sum(axis=1))
        Apes_pmt.append(regA_pmt.sum(axis=1))
        Bpes_pmt.append(regB_pmt.sum(axis=1))
        
        # Compute pes in internal and external SiPMs
        At.append(regA.shape[1])                           # length in mus of reg
        Aext_pes.append(Apes[-1][ext_sipms].mean())        # pes per SiPM in ext sipms      
        Aint_pes.append(Apes[-1][int_sipms].mean())        # pes per sipm in int sipms
        
        Bt.append(regB.shape[1])
        Bext_pes.append(Bpes[-1][ext_sipms].mean())
        Bint_pes.append(Bpes[-1][int_sipms].mean())
        Bspes.append(regB[:, :int(round(regB.shape[1] / 2.0))].sum()) # count pes in first half of regB
        Bfpes.append(np.sum(Bpes[-1]) - Bspes[-1])                    # in second half of regB
        pevts += 1
        
        if pevts == MEVTS: break
            
            
    h5rwf.close()
    #break



In [ ]:
At   =np.array(At)   
Apes =np.array(Apes)
Bpes =np.array(Bpes)
Bspes=np.array(Bspes)
Bfpes=np.array(Bfpes)
Bt   =np.array(Bt)
Aext_pes = np.array(Aext_pes) 
Aint_pes = np.array(Aint_pes) 
Bext_pes = np.array(Bext_pes) 
Bint_pes = np.array(Bint_pes)  
Apes_pmt = np.array(Apes_pmt)
Bpes_pmt = np.array(Bpes_pmt)

In [ ]:
print('NEVTS processed: ', len(At))

#### PEs in region A vs region B

In [ ]:
plt.figure(figsize=(15,5))
nza = np.nonzero(At)[0]
r=(0,.5)
plt.hist(Apes.mean(axis=1)[nza] / At[nza], bins = 20, alpha=.5, range=r, label='RegA: between S1 and S2')
plt.hist(Bpes.mean(axis=1)      / Bt     , bins = 20, alpha=.5, range=r, label='RegB: after S2')
plt.xlabel('PEs after S2 / SiPM / mus')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(15,5))
r=(-.5,.5)
plt.hist(Apes_pmt.mean(axis=1)[nza] / At[nza], bins = 20, alpha=.5, range=r, label='RegA: between S1 and S2')
plt.hist(Bpes_pmt.mean(axis=1)      / Bt     , bins = 20, alpha=.5, range=r, label='RegB: after S2')
plt.xlabel('PEs after S2 / Pmt / mus')
plt.legend()
plt.grid(True)
plt.show()

#### Cumulative waveform after S2

In [ ]:
fig, ax1 = plt.subplots(figsize=(20,5))
start = 0
stop  = -125
Bmeansum = Bswf    .sum(axis=0)[start                   :stop                     ] / float(pevts)
Bms_pmt  = Bswf_pmt.sum(axis=0)[int(start * 1000 / 25.0): int(stop * 1000 / 25.0) ] / float(pevts)
ax1.plot(range(len(Bmeansum[np.nonzero(Bmeansum)])), Bmeansum[np.nonzero(Bmeansum)] / float(nsipm), label='sipm')
ax1.set_xlabel('microseconds after s2')
ax1.set_ylabel('pes / sipm')
ax1.legend(loc=2)
ax2 = ax1.twinx()
ax2.plot(np.array(range(len(Bms_pmt [np.nonzero(Bms_pmt) ]))) / 1.0e3*25, Bms_pmt [np.nonzero(Bms_pmt )] / float(npmt), c='orange', label='pmt' )
ax2.set_ylabel('pes / pmt')
plt.title('Cumulative corrected waveforms after S2, ' + str(len(At)) + ' events')
ax2.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,3))
plt.hist(Bpes.mean(axis=1) / Bt, bins = 50, alpha=.7)
plt.title('PEs after S2 (region B)')
plt.xlabel('PEs after S2 / SiPM / mus')
plt.grid(True)
plt.show()

binmin = min((Bspes / Bt * 2 / 1792).min(), (Bfpes / Bt * 2 / 1792).min())
binmax = max((Bspes / Bt * 2 / 1792).max(), (Bfpes / Bt * 2 / 1792).max())
normed=False
plt.figure(figsize=(15,3))
plt.hist(Bspes / Bt * 2 / 1792, bins=50, normed=normed, range=(binmin, binmax), alpha=.5, label='1st  half of regB')
plt.hist(Bfpes / Bt * 2 / 1792, bins=50, normed=normed, range=(binmin, binmax), alpha=.5, label='2nd half of regB')
plt.grid(True)
plt.title('1st and 2nd half of region after S2 comparison')
plt.xlabel('PEs / SiPM / mus')
plt.legend()
plt.show()

In [ ]:
print('Here external means sipms at R >', ext_t)
binmin = min((Aint_pes[nza] / At[nza]).min(), (Aext_pes[nza] / At[nza]).min())
binmax = max((Aint_pes[nza] / At[nza]).max(), (Aext_pes[nza] / At[nza]).max())
binmin, binmax = (0.031, 0.038)
normed=False
plt.figure(figsize=(15,3))
plt.hist(Aext_pes[nza] / At[nza], bins=50, normed=normed, range=(binmin, binmax), alpha=.5, label='external sipms')
plt.hist(Aint_pes[nza] / At[nza], bins=50, normed=normed, range=(binmin, binmax), alpha=.5, label='internal sipms')
plt.grid(True)
plt.title('Between end of S1 and start of S2: RegA')
plt.xlabel('PEs Between S1 and S2 / SiPM / mus')
plt.legend()
plt.show()

#binmin = min((Bext_pes / Bt).min(), (Bint_pes / Bt).min())
#binmax = max((Bext_pes / Bt).max(), (Bint_pes / Bt).max())
normed=False
plt.figure(figsize=(15,3))
plt.hist(Bext_pes / Bt, bins=50, normed=normed, range=(binmin, binmax), alpha=.5, label='external sipms')
plt.hist(Bint_pes / Bt, bins=50, normed=normed, range=(binmin, binmax), alpha=.5, label='internal sipms')
plt.grid(True)
plt.title('After S2: RegB')
plt.xlabel('PEs after S2 / SiPM / mus')
plt.legend()
plt.show()